In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Problem1

In [20]:
train = pd.read_csv('../HW3/hw3-data/ratings_train.csv', header=None)
train.columns = ['user_id', 'movie_id', 'rating']
test = pd.read_csv('../HW3/hw3-data/ratings_test.csv', header=None)
test.columns = ['user_id', 'movie_id', 'rating']
movie = pd.read_csv('../HW3/hw3-data/movies.txt', header=None, sep='\n')
movie.columns = ['movie_name']
movie.head(10)
print(train.shape)
print(test.shape)
print(movie.shape)

(95000, 3)
(5000, 3)
(1682, 1)


In [52]:
SIGMA = 0.25
D = 10
LAMBDA = 1
N_1 = 943
N_2 = 1682
print("Number of users: ", N_1)
print("Number of movies: ", N_2)


Number of users:  943
Number of movies:  1682


In [39]:
test_M = np.empty(shape = (N_1, N_2))
test_M.fill(np.nan)
for index, row in train.iterrows():
    user_id = int(row['user_id']) - 1
    movie_id = int(row['movie_id']) - 1
    test_M[user_id][movie_id] = row['rating']

In [56]:
u_i = test_M[20]
u_i_index = [i for i, v in enumerate(u_i) if ~np.isnan(v)]
u_i_index[:10]

[0, 4, 6, 8, 14, 16, 49, 52, 55, 97]

In [63]:
np.zeros((10,10))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [69]:
u_i_index = [i for i, v in enumerate(test_M[:][20]) if ~np.isnan(v)]
u_i_index[:10]

[0, 4, 6, 8, 14, 16, 49, 52, 55, 97]

In [85]:
test = np.random.rand(3,2)
print(test)
print(np.linalg.norm(test, axis = 1))
-np.sum(np.linalg.norm(test, axis = 1))

[[0.35152532 0.43773289]
 [0.59923295 0.48560044]
 [0.07273571 0.34622945]]
[0.56140906 0.77128978 0.35378711]


-1.6864859427736443

In [106]:
test_1 = np.random.rand(1,10)
test_2 = np.random.rand(1,10)
print((test_1+test_2).shape)

(1, 10)


In [116]:
np.random.rand(1,10).reshape(10,1).shape

(10, 1)

In [121]:
class matrix_factorization:
    def __init__(self, n_1, n_2, d, n_iterations = 100) -> None:
        self.n_1 = n_1
        self.n_2 = n_2
        self.d = d
        self.n_iterations = n_iterations
        self.u = np.random.normal(0, LAMBDA, size = (n_1, self.d)) # n_1 x d
        self.v = np.random.normal(0, LAMBDA, size = (n_2, self.d)) # n_2 x d


    def init_M(self, X):
        M = np.empty(shape = (self.n_1, self.n_2))
        M.fill(np.nan)
        pair_list = []
        for index, row in X.iterrows():
            user_id = int(row['user_id']) - 1
            movie_id = int(row['movie_id']) - 1
            M[user_id][movie_id] = row['rating']
            pair_list.append((user_id, movie_id))
        return M, pair_list
        
    def fit(self, X):
        self.M, self.pair = self.init_M(X)
        log_likelihood_list = []
        
        for iter in range(self.n_iterations):
            u_temp = self.u.copy()
            v_temp = self.v.copy()
            for i in range(self.n_1):
                omega_i = [idx for idx, v in enumerate(self.M[i]) if ~np.isnan(v)]
                v_square = np.zeros((self.d, self.d)) # d x d
                M_v = np.zeros((self.d, 1)) # d x 1
                for oi in omega_i:
                    v_square += v_temp[oi].T@v_temp[oi] # d x 1 @ 1 x d
                    M_v = M_v + self.M[i][oi]*v_temp[oi].reshape(self.d, 1) # d x 1
                #  1 x d  = 1 x d @ d x d
                # print(M_v.T.shape)
                # print(np.linalg.inv((LAMBDA*SIGMA*np.eye(D)) + v_square).shape)
                self.u[i] = M_v.T @ np.linalg.inv((LAMBDA*SIGMA*np.eye(D)) + v_square)
            
            for j in range(self.n_2):
                omega_j = [idx for idx, v in enumerate(self.M[:][j]) if ~np.isnan(v)]
                u_square = np.zeros((self.d, self.d)) # d x d
                M_u = np.zeros((self.d, 1)) # d x 1
                for oj in omega_j:
                    u_square += u_temp[oj].T@u_temp[oj] # d x 1 @ 1 x d
                    M_u = M_u + self.M[oj][j]*u_temp[oj].reshape(self.d, 1) # d x 1
                #  1 x d  = 1 x d @ d x d
                self.v[i] = M_u.T @ np.linalg.inv((LAMBDA*SIGMA*np.eye(D)) + u_square)

            log_likelihood_list += self.log_likelihood()

    def log_likelihood(self):
        part1 = 0
        for user_i, movie_j in self.pair:
            part1 += np.square(self.M[user_i][movie_j] - self.u[user_i]@self.v[movie_j].T)/(2*SIGMA)
        part2 = np.sum(np.linalg.norm(self.u, axis = 1))
        part3 = np.sum(np.linalg.norm(self.v, axis = 1))
        return -part1-part2-part3

In [122]:
mf_model = matrix_factorization(N_1, N_2, D, 100)
mf_model.fit(train)

IndexError: index 943 is out of bounds for axis 0 with size 943